<a href="https://colab.research.google.com/github/anihab/dnaTokenization/blob/main/vocabulary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install Bio
!pip install tokenizers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 10.5 MB/s eta 0:00:00
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3099, in _parsed_pkg_info
    return self._pkg_info
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _pkg_info. Did you mean: 'egg_info'?

During handling of the abo

In [ ]:
# Libraries
import argparse
import gzip
import os
import csv

import pandas as pd
import numpy as np

#from Bio import SeqIO
from tokenizers import Tokenizer, models, trainers, normalizers

Train a BPE tokenizer on the full bacterial training directory to create the vocabulary file.

###Step 0

define vocabulary size and input data path
- input should be a path to a directory of bacterial fasta files

In [ ]:
# Globals
VOCAB_SIZE=4096
INPUT_PATH=""
OUTPUT_PATH=""

###Step 1

parse through all fasta files in full bacterial training directory
- to make input similar to input for pre-training, use sequences of at least 1500 nt (i.e., min sequence length = 1500)
- do not limit max sequence length

###Step 2

build vocabulary json - limit vocab size to 4096

In [ ]:
def sequence_iterator():
  '''\
  iterates through the sequences of every csv file in a given directory
  '''
  if os.path.isdir(INPUT_PATH):
        for filename in os.listdir(INPUT_PATH):
          f = os.path.join(INPUT_PATH, filename)
          with open(f, 'r') as file:
            csv_reader = csv.reader(file)
            for row in csv_reader:
              # ensure the correct number of columns exists
              if len(row) > 4:
                seq = row[4]
                # min sequence length = 1500
                if len(seq) >= 1500:
                  yield seq

def build_vocab():
  '''\
  builds a vocabulary on input
  '''
  tokenizer = Tokenizer(models.BPE())
  tokenizer.normalizer = normalizers.Sequence([normalizers.NFKC()])
  trainer = trainers.BpeTrainer(vocab_size=VOCAB_SIZE)
  tokenizer.train_from_iterator(sequence_iterator(), trainer=trainer)
  tokenizer.save(OUTPUT_PATH + "/vocabulary_" + VOCAB_SIZE + ".json")

In [ ]:
build_vocab()